In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
import time

import shapely
from shapely.geometry import Point, MultiPoint, LineString, MultiLineString

from sqlalchemy import create_engine
import traceback
import getpass

In [2]:
# clmns = ['route_id','agency_id','route_short_name','route_long_name','route_type']
routes = pd.read_csv('./data/raspis/routes.txt', sep=",")
# routes.columns = clmns

In [3]:
routes.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type
0,1,1,1,Автостанция Красная Глинка - Железнодорожный в...,3
1,14,1,38,9-й микрорайон - Юнгородок,3


In [4]:
stops = pd.read_csv('./data/raspis/stops.txt', sep=",")

In [5]:
stops.head(2)

,stop_id,stop_name,stop_lat,stop_lon
0,2,Завод Электрощит,53.381994,50.166450
1,3,пос.Южный,53.372294,50.177035


In [6]:
trips = pd.read_csv('./data/raspis/trips.txt', sep=",")

In [7]:
trips = trips.sort_values(by=['route_id', 'trip_id']).reset_index(drop=True)

In [8]:
trips.head(2)

,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,1,0_40549,0_40549_405490011_1,до Автостанция Красная Глинка,1,0_40549_H
1,1,0_40549,0_40549_405490011_2,до Железнодорожный вокзал (стоянка),0,0_40549_A


In [9]:
stop_times = pd.read_csv('./data/raspis/stop_times.txt', sep=",")

In [10]:
stop_times.head(2)

,trip_id,arrival_time,departure_time,stop_id,stop_sequence
0,0_38341_383410011_1,06:00:00,06:00:00,615,5
1,0_38341_383410011_1,06:01:00,06:01:00,554,6


In [11]:
reestr = routes.copy()

In [12]:
reestr.head(2)

,route_id,agency_id,route_short_name,route_long_name,route_type
0,1,1,1,Автостанция Красная Глинка - Железнодорожный в...,3
1,14,1,38,9-й микрорайон - Юнгородок,3


In [13]:
reestr = reestr.reset_index()
reestr['index'] = reestr['index'] + 1
reestr['city'] = 'Самара'
reestr['transp_type'] = 'автобус'
reestr['rt_type'] = 'муниципальный'
reestr = reestr[['index', 'city', 'transp_type', 
                 'rt_type', 'route_short_name', 'route_long_name', 'route_id']]
# ООО «СамараАвтоГаз»

In [14]:
reestr.columns

Index(['index', 'city', 'transp_type', 'rt_type', 'route_short_name',
       'route_long_name', 'route_id'],
      dtype='object')

In [15]:
reestr.head(2)

,index,city,transp_type,rt_type,route_short_name,route_long_name,route_id
0,1,Самара,автобус,муниципальный,1,Автостанция Красная Глинка - Железнодорожный в...,1
1,2,Самара,автобус,муниципальный,38,9-й микрорайон - Юнгородок,14


In [16]:
def stops_line(longest_trip, ind_stpid, stops):
    df_stpid = pd.DataFrame(list(longest_trip[:,ind_stpid]), columns=['stop_id'])
    df_stpid = df_stpid.merge(stops[['stop_id','stop_name']],how='left', on=['stop_id'])
    lst_stps = list(df_stpid.stop_name)
    str_stps = " – ".join(lst_stps)
    return str_stps

In [17]:
def get_stop_seq(trips, stops, rt_id, 
                 np_stptm, ind_trid, ind_arrtm, ind_deptm, ind_stpid, 
                 ind_stpseq, ind_rtid, ind_dir):
    
    one_rt = np_stptm[np_stptm[:,ind_rtid] == rt_id]
    lst_trips = list(set(list(one_rt[:,ind_trid])))
    
    lngth_max0=0
    lngth_max1=0
    if len(lst_trips) > 0:
        max_trip0 = lst_trips[0]
        max_trip1 = lst_trips[1]

        i=0
        for i in (range(len(lst_trips))):
            one_trp = one_rt[one_rt[:, ind_trid] == lst_trips[i]]
            if one_trp[0,ind_dir] == 0:
                lngth0 = len(one_trp)
                if lngth0 > lngth_max0:
                    lngth_max0 = lngth0
                    max_trip0 = lst_trips[i]
            #
            else:
                lngth1 = len(one_trp)
                if lngth1 > lngth_max1:
                    lngth_max1 = lngth1
                    max_trip1 = lst_trips[i]
            #
        longest_trip0 = one_rt[one_rt[:, ind_trid] == max_trip0]
        str_stps0 = stops_line(longest_trip0, ind_stpid, stops)
        if lngth_max1 == 0:
            longest_trip1 = []
            str_stps1 = ''
        else:
            longest_trip1 = one_rt[one_rt[:, ind_trid] == max_trip1]
            str_stps1 = stops_line(longest_trip1, ind_stpid, stops)
        # 
        return [str_stps0, str_stps1]
    else:
        print(rt_id)
        return ['', '']

In [18]:
def get_work_days(calendar, srvc_id):
    np_clnd = calendar.to_numpy()
    ind_servid = list(calendar.columns).index('service_id')
    ind_mon = list(calendar.columns).index('monday')
    ind_tue = list(calendar.columns).index('tuesday')
    ind_wed = list(calendar.columns).index('wednesday')
    ind_thu = list(calendar.columns).index('thursday')
    ind_fri = list(calendar.columns).index('friday')
    ind_sat = list(calendar.columns).index('saturday')
    ind_sun = list(calendar.columns).index('sunday')
    
    lst_days = list(np_clnd[np_clnd[:,ind_servid] == srvc_id][0,ind_mon:ind_sun+1])
    
    if 0 not in lst_days:
        descr = 'Ежедневно'
    elif ((lst_days[-2:] == [0,0]) & (lst_days[:-2] == [1,1,1,1,1])):
        descr = 'По будням'
    elif ((lst_days[-2:] == [1,1]) & (lst_days[:-2] == [0,0,0,0,0])):
        descr = 'По выходным'
    else:
        lst_words = ['понедельникам', 'вторникам', 'средам', 
         'четвергам', 'пятницам', 'субботам', 'воскресеньям']
        days_str = ''
        for j in range(len(lst_days)):
            if lst_days[j] == 1:
                days_str = days_str + ', ' + lst_words[j]
        days_str = days_str[2:]
        descr = 'По ' + days_str
    # 
    
    return descr

In [19]:
def get_work_hours(srvc_id, np_stptm, ind_trid, ind_arrtm, ind_serv):
    lst_tm = list(set(list(np_stptm[np_stptm[:,ind_serv] == srvc_id][:,ind_arrtm])))
    lst_tm.sort()
    strt_tm = lst_tm[0][:-3]
    
    last_trip_id = np_stptm[np_stptm[:,ind_arrtm] == lst_tm[-1]][0][ind_trid]
    end_tm = np_stptm[np_stptm[:,ind_trid] == last_trip_id][0,ind_arrtm]
    
    end_tm = end_tm[:-3]
    return strt_tm, end_tm

In [20]:
def get_time_in_route(rt_id, np_stptm, ind_trid, ind_arrtm, ind_rtid):
    
    one_rt = np_stptm[np_stptm[:,ind_rtid] == rt_id]
    lst_trips = list(set(list(one_rt[:,ind_trid])))
    
    ttl_tm_max = 0
    if len(lst_trips) > 0:
        i=0
        for i in (range(len(lst_trips))):
            one_trp = one_rt[one_rt[:, ind_trid] == lst_trips[i]]
            strt_tm = one_trp[0,ind_arrtm]
            end_tm = one_trp[-1,ind_arrtm]
            hour_strt = int(strt_tm[0:2])
            mnt_strt = int(strt_tm[3:5])
            hour_end = int(end_tm[0:2])
            mnt_end = int(end_tm[3:5])
            ttl_tm = (hour_end * 60 + mnt_end) - (hour_strt * 60 + mnt_strt)
            if ttl_tm > ttl_tm_max:
                ttl_tm_max = ttl_tm
            #
    else:
        print(rt_id)
    return ttl_tm_max

In [131]:
def get_rt_interval(one_rt, ind_daytrp, ind_trid, ind_deptm):
    
    lst_days_one_rt = list(set(list(one_rt[:,ind_daytrp])))

    lst_first_dep=[]

    j=0
    for j in range(len(lst_days_one_rt)):
        one_day = one_rt[one_rt[:,ind_daytrp] == lst_days_one_rt[j]]
        lst_trips = list(set(list(one_day[:,ind_trid])))

        one_day_tm=[]
        k=0
        for k in range(len(lst_trips)):
            one_trp = one_day[one_day[:, ind_trid] == lst_trips[k]]
            lst_dep = list(one_trp[:,ind_deptm])
            first_dep = lst_dep[0]
            one_day_tm.append(first_dep)
            lst_first_dep.append(one_day_tm)
    # 
    lst_first_dep.sort()
    
    lst_max_day=[]
    lst_other_interv=[]
    h=0
    for h in range(len(lst_first_dep)):
        lst1 = lst_first_dep[h]
        lst1.sort()
        lst1.reverse()
        lst_interv=[]
        g=0
        for g in range(1,len(lst1)):
            hr1,mn1,sc1 = lst1[g-1].split(":")
            hr2,mn2,sc2 = lst1[g].split(":")
            hr1,mn1,hr2,mn2 = int(hr1),int(mn1),int(hr2),int(mn2)
            if ((hr1 > 6) & (hr1 < 10)) & ((hr2 > 6) & (hr2 < 10)):
                interv = (hr1 * 60 + mn1) - (hr2 * 60 + mn2)
                lst_interv.append(interv)
            else:
                interv = (hr1 * 60 + mn1) - (hr2 * 60 + mn2)
                lst_other_interv.append(interv)
        #
        if lst_interv != []:
            max_per_day=max(lst_interv)
            lst_max_day.append(max_per_day)
    # 
    if lst_max_day == []:
        total_max_rt = max(lst_other_interv)
        total_min_rt = min(lst_other_interv)
    else:
        total_max_rt = max(lst_max_day)
        total_min_rt = min(lst_max_day)
    # 
    if total_min_rt != total_max_rt:
        rt_interv_str = str(total_min_rt) + '-' + str(total_max_rt) + ' мин.'
    else:
        rt_interv_str = str(total_min_rt) + ' мин.'
    
    return rt_interv_str

In [21]:
stop_times_rts = stop_times.merge(trips[['trip_id', 'route_id', 'direction_id', 'service_id']], 
                                   how='left', on=['trip_id'])
stop_times_rts['day_trips_id'] = stop_times_rts['trip_id'].apply(lambda one: "_".join(one.split("_")[:-1]))

In [22]:
np_stptm = stop_times_rts.to_numpy()
ind_trid = list(stop_times_rts.columns).index('trip_id')
ind_arrtm = list(stop_times_rts.columns).index('arrival_time')
ind_deptm = list(stop_times_rts.columns).index('departure_time')
ind_stpid = list(stop_times_rts.columns).index('stop_id')
ind_stpseq = list(stop_times_rts.columns).index('stop_sequence')
ind_rtid = list(stop_times_rts.columns).index('route_id')
ind_dir = list(stop_times_rts.columns).index('direction_id')
ind_serv = list(stop_times_rts.columns).index('service_id')
ind_daytrp = list(stop_times_rts.columns).index('day_trips_id')

In [23]:
lst_rtid=list(routes.route_id.unique())
lst_rt_two_seq = []

# for j in tqdm(range(2)):
for j in tqdm(range(len(lst_rtid))):
    rt_id = lst_rtid[j]
    lst_stp_seq = get_stop_seq(trips, stops, rt_id, 
                     np_stptm, ind_trid, ind_arrtm, ind_deptm, ind_stpid, 
                     ind_stpseq, ind_rtid, ind_dir)
    #
    lst_rt_two_seq.append([rt_id] + lst_stp_seq)
# 

177
189
282



In [24]:
df_stop_seq = pd.DataFrame(lst_rt_two_seq, columns=['route_id', 'stop_seq_frwr', 'stop_seq_bck'])

In [25]:
df_stop_seq.head(2)

,route_id,stop_seq_frwr,stop_seq_bck
0,1,Автостанция Красная Глинка – Завод Электрощит ...,Железнодорожный вокзал (стоянка) – Железнодоро...
1,14,9-й микрорайон – Кинотеатр Шипка – 10-й микрор...,Юнгородок – ОАО Гидроавтоматика – Авиагородок ...


In [26]:
# calendar

In [27]:
calendar = pd.read_csv('./data/raspis/calendar.txt', sep=",")

In [28]:
calendar.head(2)

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date
0,0_38341,1,1,1,1,1,0,0,20160201,20210916
1,0_38342,0,0,0,0,0,1,1,20160206,20210916


In [29]:
lst_srvc_id = list(calendar.service_id.unique())
lst_ttl_wrk_tm = []
i=0
for srvc_id in tqdm(lst_srvc_id):
    descr = get_work_days(calendar, srvc_id)
    try:
        strt_tm, end_tm = get_work_hours(srvc_id, np_stptm, ind_trid, ind_arrtm, ind_serv)
    except:
        strt_tm, end_tm = '', ''
        print(srvc_id)
    total_tm = descr + ': c ' + strt_tm + ' до ' + end_tm
    lst_ttl_wrk_tm.append([srvc_id, total_tm])
# 

0_39579
0_39795
0_40050
0_40087



In [30]:
df_calend = pd.DataFrame(lst_ttl_wrk_tm, columns=['service_id', 'work_time'])

In [31]:
serv_rts = stop_times_rts[['service_id', 'route_id']].copy()
serv_rts = serv_rts.drop_duplicates().reset_index(drop=True)

In [32]:
df_calend = df_calend.merge(serv_rts, 
                            how='left', on=['service_id'])

In [33]:
df_calend.head(2)

,service_id,work_time,route_id
0,0_38341,По будням: c 06:00 до 20:30,199.0
1,0_38342,По выходным: c 06:00 до 20:10,199.0


In [34]:
df_calend2=df_calend.copy()

In [35]:
grp_df_calend2 = df_calend2.groupby(['route_id']).agg({'work_time':'sum'})

In [36]:
grp_df_calend2 = grp_df_calend2.reset_index()

In [140]:
grp_df_calend2['work_time'] = grp_df_calend2['work_time'].apply(lambda s: s.replace("По", " По"))

In [141]:
grp_df_calend2.head(2)

,route_id,work_time
0,1.0,По выходным: c 05:27 до 22:00 По будням: c 05...
1,14.0,По выходным: c 06:20 до 20:03 По будням: c 05...


In [38]:
lst_rtid=list(routes.route_id.unique())
lst_rt_two_seq = []

# for j in tqdm(range(2)):
for j in tqdm(range(len(lst_rtid))):
    rt_id = lst_rtid[j]
    ttl_tm_max1 = get_time_in_route(rt_id, np_stptm, ind_trid, ind_arrtm, ind_rtid)
    if ttl_tm_max1 / 60 < 1:
        ttl_tm_max2 = str(ttl_tm_max1) + ' мин.'
    else:
        tmp = int(ttl_tm_max1 / 60)
        ttl_tm_max2 = str(tmp) + ' ч. ' + str(ttl_tm_max1 - (tmp * 60)) + ' мин.'
    # 
    lst_rt_two_seq.append([rt_id, ttl_tm_max2, ttl_tm_max1])
# 

177
189
282



In [39]:
df_tm_in_rt = pd.DataFrame(lst_rt_two_seq, columns=['route_id', 'time_str', 'time_min'])

In [40]:
df_tm_in_rt.head(3)

,route_id,time_str,time_min
0,1,1 ч. 23 мин.,83
1,14,36 мин.,36
2,15,1 ч. 6 мин.,66


In [41]:
rt_id=1
one_rt = np_stptm[np_stptm[:,ind_rtid] == rt_id]

In [123]:
rt_interv_str = get_rt_interval(one_rt, ind_daytrp, ind_trid, ind_deptm)

In [124]:
rt_interv_str

'39-97 мин.'

In [132]:
lst_rtid=list(routes.route_id.unique())
lst_rt_intervs = []

for t in tqdm(range(len(lst_rtid))):
    rt_id = lst_rtid[t]
    one_rt = np_stptm[np_stptm[:,ind_rtid] == rt_id]
    try:
        rt_interv_str = get_rt_interval(one_rt, ind_daytrp, ind_trid, ind_deptm)
    except:
        print(rt_id)
        rt_interv_str = ''
    lst_rt_intervs.append([rt_id, rt_interv_str])
# 

177
189
282



In [133]:
df_intervs = pd.DataFrame(lst_rt_intervs, columns=['route_id', 'interval'])

In [134]:
df_intervs.head()

,route_id,interval
0,1,39-97 мин.
1,14,34-46 мин.
2,15,42-108 мин.
3,17,39-93 мин.
4,18,45 мин.


In [ ]:
# reestr - названия
# df_stop_seq - трассы
# grp_df_calend2 - даты работы
# df_tm_in_rt - время в пути
# df_intervs - интервалы

In [135]:
print(len(reestr))
print(len(df_stop_seq))
print(len(grp_df_calend2))
print(len(df_tm_in_rt))
print(len(df_intervs))

88
88
85
88
88


In [136]:
reestr.head(2)

,index,city,transp_type,rt_type,route_short_name,route_long_name,route_id
0,1,Самара,автобус,муниципальный,1,Автостанция Красная Глинка - Железнодорожный в...,1
1,2,Самара,автобус,муниципальный,38,9-й микрорайон - Юнгородок,14


In [151]:
print(len(reestr))
reestr_fin = reestr.merge(df_stop_seq, how='left', on=['route_id'])
reestr_fin = reestr_fin.merge(grp_df_calend2, how='left', on=['route_id'])
reestr_fin = reestr_fin.merge(df_intervs, how='left', on=['route_id'])
reestr_fin['vehicl_amnt'] = None
reestr_fin = reestr_fin.merge(df_tm_in_rt, how='left', on=['route_id'])
reestr_fin['route_length'] = None
reestr_fin['agency'] = 'ООО «СамараАвтоГаз»'
reestr_fin['vehicl_type'] = None
print(len(reestr_fin))

88
88


In [155]:
reestr_fin['route_id'] = None

In [159]:
dct_renm = {'index':'№',
'city':'Город',
'transp_type':'Тип транспорта',
'rt_type':'Тип маршрута',
'route_short_name':'Номер маршрута',
'route_long_name':'Наименование маршрута',
'route_id':'Тип подвижного состава на маршруте (основной, данные реестра)',
'stop_seq_frwr':'Трасса следования по маршруту регулярных перевозок, прямое направление',
'stop_seq_bck':'Трасса следования по маршруту регулярных перевозок, обратное направление',
'work_time':'Режим работы маршрута',
'interval':'Интервалы движения на маршруте',
'vehicl_amnt':'Выпуск на маршруте (из открытых источников)',
'time_str':'Время рейса в одном направлении',
'time_min':'Время рейса в одном направлении, мин',
'route_length':'Протяженность (км) маршрута регулярных перевозок',
'agency':'Наименование предприятия, осуществляющего перевозки по маршруту регулярных перевозок',
'vehicl_type':'Подвижной состав (из открытых источников)'
}

In [160]:
reestr_fin = reestr_fin.rename(columns=dct_renm)

In [161]:
reestr_fin.head(2)

,№,Город,Тип транспорта,Тип маршрута,Номер маршрута,Наименование маршрута,"Тип подвижного состава на маршруте (основной, данные реестра)","Трасса следования по маршруту регулярных перевозок, прямое направление","Трасса следования по маршруту регулярных перевозок, обратное направление",Режим работы маршрута,Интервалы движения на маршруте,Выпуск на маршруте (из открытых источников),Время рейса в одном направлении,"Время рейса в одном направлении, мин",Протяженность (км) маршрута регулярных перевозок,"Наименование предприятия, осуществляющего перевозки по маршруту регулярных перевозок",Подвижной состав (из открытых источников)
0,1,Самара,автобус,муниципальный,1,Автостанция Красная Глинка - Железнодорожный в...,None,Автостанция Красная Глинка – Завод Электрощит ...,Железнодорожный вокзал (стоянка) – Железнодоро...,По выходным: c 05:27 до 22:00 По будням: c 05...,39-97 мин.,None,1 ч. 23 мин.,83,None,ООО «СамараАвтоГаз»,None
1,2,Самара,автобус,муниципальный,38,9-й микрорайон - Юнгородок,None,9-й микрорайон – Кинотеатр Шипка – 10-й микрор...,Юнгородок – ОАО Гидроавтоматика – Авиагородок ...,По выходным: c 06:20 до 20:03 По будням: c 05...,34-46 мин.,None,36 мин.,36,None,ООО «СамараАвтоГаз»,None


In [163]:
reestr_fin.to_excel("./data/reestr_fin.xlsx", index=False)